In [ ]:
import gym
import minerl
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.torch_layers import NatureCNN
from minerl.herobraine.env_specs.human_survival_specs import HumanSurvival


ImportError: cannot import name 'simple_actions' from 'minerl.herobraine.wrappers' (/data/homezvol0/gaog5/.conda/envs/rl2/lib/python3.9/site-packages/minerl/herobraine/wrappers/__init__.py)

In [17]:
env = gym.make("MineRLObtainDiamondShovel-v0")

print(env.observation_space)
print(env.action_space)

Dict(inventory:Dict(acacia_boat:Box(low=0, high=2304, shape=()), acacia_button:Box(low=0, high=2304, shape=()), acacia_door:Box(low=0, high=2304, shape=()), acacia_fence:Box(low=0, high=2304, shape=()), acacia_fence_gate:Box(low=0, high=2304, shape=()), acacia_leaves:Box(low=0, high=2304, shape=()), acacia_log:Box(low=0, high=2304, shape=()), acacia_planks:Box(low=0, high=2304, shape=()), acacia_pressure_plate:Box(low=0, high=2304, shape=()), acacia_sapling:Box(low=0, high=2304, shape=()), acacia_sign:Box(low=0, high=2304, shape=()), acacia_slab:Box(low=0, high=2304, shape=()), acacia_stairs:Box(low=0, high=2304, shape=()), acacia_trapdoor:Box(low=0, high=2304, shape=()), acacia_wood:Box(low=0, high=2304, shape=()), activator_rail:Box(low=0, high=2304, shape=()), air:Box(low=0, high=2304, shape=()), allium:Box(low=0, high=2304, shape=()), ancient_debris:Box(low=0, high=2304, shape=()), andesite:Box(low=0, high=2304, shape=()), andesite_slab:Box(low=0, high=2304, shape=()), andesite_sta

In [18]:
import numpy as np
from gym import ObservationWrapper
from gym.spaces import Box

class MineRLObservationWrapper(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # Grayscale image: 64x64
        self.observation_space = Box(low=0, high=255, shape=(1, 64, 64), dtype=np.uint8)

    def observation(self, obs):
        pov = obs['pov']
        gray_pov = np.mean(pov, axis=2).astype(np.uint8)  # Convert to grayscale
        gray_pov = np.expand_dims(gray_pov, axis=0)  # (1, 64, 64)
        return gray_pov

env = MineRLObservationWrapper(env)


In [19]:
from gym import ActionWrapper
from gym.spaces import Discrete

class MineRLActionWrapper(ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        self._actions = [
            {'forward': 1},                               # Move forward
            {'attack': 1},                                # Mine/attack
            {'jump': 1},                                  # Jump
            {'craft': 'planks'},                          # Craft planks
            {'craft': 'stick'},                           # Craft sticks
            {'craft': 'crafting_table'},                  # Craft crafting table
            {'craft': 'wooden_pickaxe'},                  # Craft wooden pickaxe
            {'craft': 'stone_pickaxe'},                   # Craft stone pickaxe
            {'craft': 'furnace'},                         # Craft furnace
            {'craft': 'iron_pickaxe'},                    # Craft iron pickaxe
            {'craft': 'diamond_shovel'},                  # Craft diamond shovel
            {'equip': 'wooden_pickaxe'},                  # Equip wooden pickaxe
            {'equip': 'stone_pickaxe'},                   # Equip stone pickaxe
            {'equip': 'iron_pickaxe'},                    # Equip iron pickaxe
            {'camera': [0, 5]},                           # Look right
            {'camera': [0, -5]},                          # Look left
        ]
        self.action_space = Discrete(len(self._actions))

    def action(self, action_idx):
        return self._actions[action_idx]

env = MineRLActionWrapper(env)


In [20]:
import torch as th
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3 import PPO

class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU(),
            nn.Flatten()
        )
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations):
        return self.linear(self.cnn(observations))


In [21]:
from stable_baselines3.common.vec_env import DummyVecEnv

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256),
)

env = DummyVecEnv([lambda: env])

model = PPO(
    'CnnPolicy',
    env,
    verbose=1,
    policy_kwargs=policy_kwargs,
    tensorboard_log="./minerl_tensorboard/"
)


/data/homezvol0/gaog5/.conda/envs/rl2/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [22]:
# Train for an initial short run to test
model.learn(total_timesteps=10000)

# Save the model
model.save("minerl_diamond_shovel_ppo")


/data/homezvol0/gaog5/.conda/envs/rl2/lib/python3.9/runpy.py:127: RuntimeWarning: 'minerl.utils.process_watcher' found in sys.modules after import of package 'minerl.utils', but prior to execution of 'minerl.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


EOFError: /data/homezvol0/gaog5/.conda/envs/rl2/lib/python3.9/site-packages/minerl/env/../MCP-Reborn
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
[23:48:09] [Render thread/INFO]: Environment: authHost='https://authserver.mojang.com', accountsHost='https://api.mojang.com', sessionHost='https://sessionserver.mojang.com', servicesHost='https://api.minecraftservices.com', name='PROD'
[23:48:13] [Render thread/INFO]: Setting user: Player39
[23:48:13] [Render thread/INFO]: Backend library: LWJGL version 3.2.2 SNAPSHOT
---- Minecraft Crash Report ----
// Uh... Did I do that?

Time: 2/20/25 11:48 PM
Description: Initializing game

java.lang.IllegalStateException: Failed to initialize GLFW, errors: GLFW error during init: [0x10008]139932725251840
	at com.mojang.blaze3d.platform.GLX._initGlfw(GLX.java:80)
	at com.mojang.blaze3d.syste

In [ ]:
obs = env.reset()
done = False
total_reward = 0

while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    env.render()

print("Total reward:", total_reward)
